## Deliverable 2. Create a Customer Travel Destinations Map.

In [98]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [99]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("D:/World_Weather_Analysis/Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Belozerskoye,RU,55.8214,65.5842,54.99,66,5,4.81,clear sky
1,1,Mahebourg,MU,-20.4081,57.7000,72.27,81,28,3.36,scattered clouds
2,2,Rawson,AR,-43.3002,-65.1023,45.37,56,0,12.03,clear sky
3,3,Hermanus,ZA,-34.4187,19.2345,60.91,44,7,8.59,clear sky
4,4,Bluff,NZ,-46.6000,168.3333,49.80,71,74,5.77,broken clouds


In [100]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 85


In [101]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]

In [102]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                279
City                   279
Country                277
Lat                    279
Lng                    279
Max Temp               279
Humidity               279
Cloudiness             279
Wind Speed             279
Current Description    279
dtype: int64

In [103]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.count()

City_ID                277
City                   277
Country                277
Lat                    277
Lng                    277
Max Temp               277
Humidity               277
Cloudiness             277
Wind Speed             277
Current Description    277
dtype: int64

In [104]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Mahebourg,MU,72.27,scattered clouds,-20.4081,57.7000,
5,Albany,US,79.45,clear sky,42.6001,-73.9662,
6,Kavieng,PG,82.90,moderate rain,-2.5744,150.7967,
8,Rikitea,PF,76.12,clear sky,-23.1203,-134.9692,
11,San Patricio,US,83.12,clear sky,28.0170,-97.5169,
18,Yarmouth,US,77.22,broken clouds,41.7057,-70.2286,
19,Bambous Virieux,MU,72.37,scattered clouds,-20.3428,57.7575,
21,Eydhafushi,MV,82.98,overcast clouds,5.1033,73.0708,
23,Manokwari,ID,82.27,overcast clouds,-0.8667,134.0833,
25,Champasak,LA,78.08,overcast clouds,14.7500,106.0000,


In [105]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [110]:
# 7. Drop the rows where there is no Hotel Name.
# The rows with no hotel names are not being recognized as empty, replaced the empty value with np.nan.
hotel_df['Hotel Name'] = hotel_df['Hotel Name'].replace("",np.nan)
# Then dropna will recognize them as an empty hotel name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.count()

City                   260
Country                260
Max Temp               260
Current Description    260
Lat                    260
Lng                    260
Hotel Name             260
dtype: int64

In [111]:
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Mahebourg,MU,72.27,scattered clouds,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa
6,Kavieng,PG,82.90,moderate rain,-2.5744,150.7967,Nusa Island Retreat
8,Rikitea,PF,76.12,clear sky,-23.1203,-134.9692,Pension Maro'i
11,San Patricio,US,83.12,clear sky,28.0170,-97.5169,"Motel 6 Sinton, TX"
18,Yarmouth,US,77.22,broken clouds,41.7057,-70.2286,Blue Rock Resort
...,...,...,...,...,...,...,...
686,Siocon,PH,78.71,overcast clouds,7.7061,122.1353,ABS Hotel
687,Makakilo City,US,84.52,overcast clouds,21.3469,-158.0858,Marriott's Ko Olina Beach Club
688,Mae Hong Son,TH,77.14,broken clouds,19.3003,97.9685,Baiyoke Chalet Hotel
689,Ilhabela,BR,70.52,clear sky,-23.7781,-45.3581,Ilha Flat Hotel


In [112]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [113]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
# format of the info box on the map figure.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp}°F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [114]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))